# 1. 把底模safetensors转化为文件夹权重模式

在这里下载底模https://civitai.com/models/27259?modelVersionId=221220然后移动到文件夹

In [ ]:
!wget --content-disposition "https://civitai.com/api/download/models/221220?type=Model&format=SafeTensor&size=pruned&fp=fp16"

In [ ]:
from diffusers import StableDiffusionPipeline

checkpoint_path = r"tmndMix_tmndMixSPRAINBOW.safetensors"
save_path = r"basemodel"
# 加载 .safetensors 文件
pipeline = StableDiffusionPipeline.from_single_file(checkpoint_path)

# 将模型保存为 diffusers 格式
pipeline.save_pretrained(save_path)

# 2. 把数据集解压

In [ ]:
!apt-get install unrar
!unrar x pic.rar -o+ /

# 3. 训练

In [ ]:
!git clone https://github.com/sodabreak/PoseCtrl.git
!cd poseCtrl

单卡

In [ ]:
python train_colab_V2.py --pretrained_model_name_or_path "/basemodel" --base_point_path "/PoseCtrl/dataSet/standardVertex.txt" --data_root_path "/pic" --train_batch_size 4 --save_steps 2000 

多卡

- 先运行配置
- 回答问题：使用混合精度fp16

In [ ]:
accelerate config

In [ ]:
accelerate launch --num_processes 2 --multi_gpu --mixed_precision "fp16" train_colab_V2.py --pretrained_model_name_or_path "/basemodel" --base_point_path "/PoseCtrl/dataSet/standardVertex.txt" --data_root_path "/pic" --train_batch_size 4 --save_steps 2000 

# 权重处理

所有训练得到的权重都在/sd-pose_ctrl文件夹内

把模型大权重转化为训练部分小权重   只要转换model.pth就行，其他权重没用  运行得到posectrl.bin

权重处理和inference都能把训练权重下载下来，直接在本机直接运行

In [ ]:
""" V2 """
import torch
import torch
from pathlib import Path
import os
def change_checkpoint(checkpoint_path, new_checkpoint_path):
    sd = torch.load(checkpoint_path, map_location="cpu")
    image_proj_model_point_sd = {}
    atten_sd = {}
    proj_sd={}
    for k in sd:
        if k.startswith("unet"):
            pass
        elif k.startswith("image_proj_model_point"):
            image_proj_model_point_sd[k.replace("image_proj_model_point.", "")] = sd[k]
        elif k.startswith("atten_modules"):
            atten_sd[k.replace("atten_modules.", "")] = sd[k]
        elif k.startswith("image_proj_model"):
            proj_sd[k.replace("image_proj_model.", "")] = sd[k]
    os.makedirs(new_checkpoint_path, exist_ok=True)
    new_checkpoint_path = Path(new_checkpoint_path, "posectrl.bin")
    torch.save({"image_proj_model_point": image_proj_model_point_sd, "atten_modules": atten_sd, "image_proj_model": proj_sd}, new_checkpoint_path)
    print(f"Saved new checkpoint to {new_checkpoint_path}")

# 修改模型路径
ckpt = r""

change_checkpoint(ckpt, r"/result")

# inference

In [ ]:
""" V2 """
import torch
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler, AutoencoderKL
from PIL import Image
import sys
import os
notebook_path = os.getcwd()
sys.path.append(notebook_path)
sys.path.append(os.path.join(notebook_path, "poseCtrl"))
from poseCtrl.models.pose_adaptor import VPmatrixPoints, ImageProjModel
from poseCtrl.models.attention_processor import AttnProcessor, PoseAttnProcessor
from poseCtrl.data.dataset import CustomDataset, load_base_points
from poseCtrl.models.posectrl import PoseCtrl,PoseCtrlV1,PoseCtrlV2
import numpy as np

# 加载模型点
base_point_path=r'\dataSet\standardVertex.txt'
raw_base_points=load_base_points(base_point_path)  
# 底模路径
base_model_path = r"F:\Projects\diffusers\ProgramData\basemodel"
vae_model_path = "stabilityai/sd-vae-ft-mse"
image_encoder_path = "laion/CLIP-ViT-H-14-laion2B-s32B-b79K"
# 上一步生成的模型权重路径
ip_ckpt = r"F:\Projects\diffusers\Project\sd-pose_ctrl\V2\posectrl.bin"
# 测试集图片路径
path = r"F:\\Projects\\diffusers\\ProgramData\\sample_new"
device = "cuda"

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

noise_scheduler = DDIMScheduler(
    num_train_timesteps=1000,
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,
    set_alpha_to_one=False,
    steps_offset=1,
)
vae = AutoencoderKL.from_pretrained(vae_model_path).to(dtype=torch.float16)

# load SD pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    scheduler=noise_scheduler,
    vae=vae,
    feature_extractor=None,
    safety_checker=None
)

dataset = CustomDataset(path)

# 选择一张图片
data = dataset[61]
from torchvision import transforms

transform = transforms.Resize((256, 256))

image = data['image']
image_pil = transforms.ToPILImage()(image)
image_pil = transform(image_pil)  

g_image = data['feature']
g_image_pil = transforms.ToPILImage()(g_image)
g_image_pil = transform(g_image_pil) 

vmatrix = data['view_matrix'].to(torch.float16).unsqueeze(0).to(device)
pmatrix = data['projection_matrix'].to(torch.float16).unsqueeze(0).to(device)

pose_model = PoseCtrlV2(pipe, image_encoder_path, ip_ckpt, raw_base_points, device)
images = pose_model.generate(pil_image=g_image, num_samples=4, num_inference_steps=50, seed=42, V_matrix=vmatrix, P_matrix=pmatrix )
grid = image_grid(images, 1, 4)
grid